In [1]:
import keras
import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation, \
    Flatten, Lambda, LSTM, RepeatVector, TimeDistributed, Reshape, \
    Conv2D, MaxPooling2D, BatchNormalization, ConvLSTM2D, Bidirectional, Masking
import keras.callbacks as Callbacks 
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import scale
from matplotlib import pyplot as plt

/usr/share/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from skimage.util import random_noise
from skimage.transform import rotate
from scipy import ndimage

In [3]:
data = h5py.File("data/data_sample.hdf5", "r")
print("data keys: " + str(list(data.keys())))

data keys: ['images', 'labels', 'observation_days']


In [4]:
images = data["images"][:]
print("image shape: (sample, x_size, y_size, epoch) = " + str(images.shape))
labels = data["labels"][:]

image shape: (sample, x_size, y_size, epoch) = (72000, 21, 21, 48)


In [5]:
def preprocess_img(images, percent=0.25):
    
    size, timestep, _, _ = images.shape
    sample_size = int(size * percent)
    
    np.random.seed(209)
    # pick the sequence with replacement
    sample_seq_indices = np.random.choice(np.arange(sample_size), sample_size)
    # pick the timestep
    sample_inseq_indices = np.random.choice(np.arange(timestep), sample_size)
    # pick the aug mean
    sample_aug_type = np.random.choice(np.arange(4), sample_size)
    
    for idx in range(sample_size):
        i, j, k = sample_seq_indices[idx], \
            sample_inseq_indices[idx], sample_aug_type[idx]
        if k == 0: # rotate
            images[i, j] = rotate(images[i, j], 45)
        elif k == 1: # random noise
            images[i, j] = random_noise(images[i, j], seed=209)
        elif k == 2: # horizontal flip
            images[i, j] = images[i, j][:,::-1]
        else: # vertical flip
            images[i, j] = images[i, j][::-1,:]

In [6]:
def scale_images(images):
    t_images = np.transpose(images, (0,3,1,2))
    rt_images = t_images.reshape(72000*48, 21, 21)
    max_per_img = np.max(rt_images.reshape(-1, 21*21), axis=1, keepdims=1)
    scaled_images = rt_images.reshape(-1, 21*21) / max_per_img
    scaled_images = scaled_images.reshape(-1, 21, 21).reshape(-1, 48, 21, 21)
    return scaled_images

In [7]:
def txt2digit(labels):
    dic = {'Asteroids':0, 'Constant':1, 'EmptyLigh':2, 'M33Cephei':3, 'RRLyrae':4, 'Supernova':5}
    labels_digit = np.array([dic[i] for i in labels])
    return labels_digit

In [8]:
def build_dataset(images, labels):
    scaled_img = scale_images(images)
    preprocess_img(scaled_img)
    x = np.expand_dims(scaled_img, len(scaled_img.shape))
    y = to_categorical(txt2digit(labels))
    return x, y

In [9]:
x, y = build_dataset(images, labels)

In [10]:
def init_model(cnn_input_dim=21, cnn_output_dim=128, cnn_dropout=0.5,
               rnn_hidden_dim=128, rnn_output_dim=64, num_classes=6, rnn_dropout=0.5, timestep=48):
    
    
    intput_shape = (timestep, cnn_input_dim, cnn_input_dim, 1)
    model = Sequential()
    # CNN
    model.add(TimeDistributed(Conv2D(32, (4,4), \
                     padding='same', activation='relu', kernel_initializer='uniform'), \
                              input_shape=intput_shape))
    
    model.add(BatchNormalization())
    # model.add(Dropout(cnn_dropout))
    
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1))))
    model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu', kernel_initializer='uniform')))
    
    #model.add(BatchNormalization())
    
    model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu', kernel_initializer='uniform')))
    
    #model.add(BatchNormalization())
    
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1))))
    model.add(TimeDistributed(Conv2D(32, (3,3), padding='same', activation='relu', kernel_initializer='uniform')))
    
    #model.add(BatchNormalization())
    
    model.add(TimeDistributed(Conv2D(32, (3,3), padding='same', activation='relu', kernel_initializer='uniform')))
    
    #model.add(BatchNormalization())
#     model.add(Dropout(cnn_dropout))
    
    model.add(TimeDistributed(Conv2D(32, (3,3), padding='same', activation='relu', kernel_initializer='uniform')))
    #model.add(Dropout(cnn_dropout))
    model.add(BatchNormalization())
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1))))
    model.add(TimeDistributed(Flatten()))
    model.add(TimeDistributed(Dense(128, activation='relu')))
    
    #model.add(BatchNormalization())
#     model.add(Dropout(cnn_dropout))
    
    model.add(TimeDistributed(Dense(cnn_output_dim, activation='relu')))
    
    model.add(Bidirectional(LSTM(rnn_hidden_dim, dropout=rnn_dropout, return_sequences=True), \
                          input_shape=(timestep, cnn_output_dim)))
    # repeat vector for timestep
    #model.add(RepeatVector(timestep))
    # decode
    model.add(Bidirectional(LSTM(rnn_hidden_dim, dropout=rnn_dropout)))

    model.add(Dense(num_classes, activation='softmax'))
    
    #out = rnn(cat)
    
    return model#Model(inputs=iL, outputs=out)

In [11]:
def init_model3(cnn_input_dim=21, cnn_output_dim=128, cnn_dropout=0.5,
               rnn_hidden_dim=128, rnn_output_dim=64, num_classes=6, rnn_dropout=0.5, timestep=48,
               bidir_mode='concat'):
    
    input_shape = (timestep, cnn_input_dim, cnn_input_dim, 1, )
    
    model = Sequential()
    
    model.add(TimeDistributed(Masking(), input_shape=input_shape, name='Masking'))
    model.add(TimeDistributed(Conv2D(48, (4,4), \
                     padding='same', activation='relu', kernel_initializer='uniform'), \
                              input_shape=input_shape, name='Conv2D_1'))
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1)), name='MaxPooling2D_1'))
    
    model.add(TimeDistributed(Conv2D(24, (3,3), padding='same', activation='relu', kernel_initializer='uniform'),\
                 name='Conv2D_2'))
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1)), name='MaxPooling2D_2'))

    model.add(TimeDistributed(Conv2D(12, (3,3), padding='same', activation='relu', kernel_initializer='uniform'), \
                             name='Conv2D_3'))
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1)), name='MaxPooling2D_3'))
    
    model.add(TimeDistributed(Flatten(), name='Faltten'))

    model.add(TimeDistributed(Dense(128, activation='relu'), name='Dense_128'))
    model.add(Dropout(0.25))
    
    model.add(Bidirectional(LSTM(rnn_hidden_dim, dropout=rnn_dropout, return_sequences=True), \
                          input_shape=(timestep, cnn_output_dim), merge_mode=bidir_mode, name='Bi-directional_LSTM_1'))
    model.add(Bidirectional(LSTM(rnn_hidden_dim, dropout=rnn_dropout), merge_mode=bidir_mode, name='Bi-directional_LSTM_2'))
    model.add(Dense(6, activation='softmax', name='Output_Dense'))
    
    return model

In [12]:
model = init_model3()

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Masking (TimeDistributed)    (None, 48, 21, 21, 1)     0         
_________________________________________________________________
Conv2D_1 (TimeDistributed)   (None, 48, 21, 21, 48)    816       
_________________________________________________________________
MaxPooling2D_1 (TimeDistribu (None, 48, 19, 19, 48)    0         
_________________________________________________________________
Conv2D_2 (TimeDistributed)   (None, 48, 19, 19, 24)    10392     
_________________________________________________________________
MaxPooling2D_2 (TimeDistribu (None, 48, 17, 17, 24)    0         
_________________________________________________________________
Conv2D_3 (TimeDistributed)   (None, 48, 17, 17, 12)    2604      
_________________________________________________________________
MaxPooling2D_3 (TimeDistribu (None, 48, 15, 15, 12)    0         
__________

In [14]:
batch_size = 80
opt = keras.optimizers.Adam(lr=5*1e-4)

In [15]:
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
cpt = Callbacks.ModelCheckpoint(filepath='models/model3_#lstm=2_bi=concat_weights.{epoch:02d}-{val_loss:.2f}.hdf5', 
                      save_best_only=True, period=10)
tensorboard = Callbacks.TensorBoard(log_dir='./logs')

In [17]:
model.fit(x[:1000], y[:1000], batch_size=256, epochs=1,
              validation_split=0.2, shuffle=False, callbacks=[cpt, tensorboard])

Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 11s 14ms/step - loss: 1.7918 - acc: 0.1712 - val_loss: 1.7965 - val_acc: 0.1300


In [135]:
model.fit(x[::2], y[::2], batch_size=80, epochs=100,
              validation_split=0.2, shuffle=True, callbacks=[cpt])

Train on 28800 samples, validate on 7200 samples
Epoch 1/100
28800/28800 [==============================] - 241s 8ms/step - loss: 1.5522 - acc: 0.3018 - val_loss: 1.5422 - val_acc: 0.3571
Epoch 2/100
28800/28800 [==============================] - 234s 8ms/step - loss: 1.2763 - acc: 0.3983 - val_loss: 1.0378 - val_acc: 0.4697
Epoch 3/100
28800/28800 [==============================] - 234s 8ms/step - loss: 1.0860 - acc: 0.4692 - val_loss: 0.9510 - val_acc: 0.5094
Epoch 4/100
28800/28800 [==============================] - 234s 8ms/step - loss: 0.9479 - acc: 0.5215 - val_loss: 0.8956 - val_acc: 0.5414
Epoch 5/100
28800/28800 [==============================] - 234s 8ms/step - loss: 0.8940 - acc: 0.5459 - val_loss: 0.8682 - val_acc: 0.5553
Epoch 6/100
28800/28800 [==============================] - 234s 8ms/step - loss: 0.8925 - acc: 0.5502 - val_loss: 0.8542 - val_acc: 0.5601
Epoch 7/100
28800/28800 [==============================] - 234s 8ms/step - loss: 0.8382 - acc: 0.5653 - val_loss: 0.8

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



28800/28800 [==============================] - 233s 8ms/step - loss: 0.6155 - acc: 0.7075 - val_loss: 0.6817 - val_acc: 0.6883
Epoch 39/100
28800/28800 [==============================] - 233s 8ms/step - loss: 0.6228 - acc: 0.7068 - val_loss: 0.5999 - val_acc: 0.7204
Epoch 40/100
28800/28800 [==============================] - 233s 8ms/step - loss: 0.6132 - acc: 0.7130 - val_loss: 0.6027 - val_acc: 0.7275
Epoch 41/100
28800/28800 [==============================] - 233s 8ms/step - loss: 0.5971 - acc: 0.7219 - val_loss: 0.5885 - val_acc: 0.7372
Epoch 42/100
28800/28800 [==============================] - 232s 8ms/step - loss: 0.6036 - acc: 0.7183 - val_loss: 0.5659 - val_acc: 0.7360
Epoch 43/100
28800/28800 [==============================] - 233s 8ms/step - loss: 0.5810 - acc: 0.7289 - val_loss: 0.5745 - val_acc: 0.7314
Epoch 44/100
28800/28800 [==============================] - 233s 8ms/step - loss: 0.5752 - acc: 0.7325 - val_loss: 0.5707 - val_acc: 0.7406
Epoch 45/100
28800/28800 [=======

In [183]:
tensorboard = Callbacks.TensorBoard(log_dir='./logs')

In [47]:
model3.fit(stateful_x, stateful_y, batch_size=1, epochs=100,
              validation_split=0.2, shuffle=False, callbacks=[cpt, ResetStatesCallback()])

Train on 288000 samples, validate on 72000 samples
Epoch 1/100
   776/288000 [..............................] - ETA: 15:38:51 - loss: 1.6114 - acc: 0.3918

KeyboardInterrupt: 